# Google - Google Pay Digital Wallet Transaction Security Patterns

```SQL

CREATE TABLE fct_transactions (
    transaction_id INTEGER,
    merchant_category VARCHAR,
    transaction_status VARCHAR,
    transaction_date DATE
);

INSERT INTO fct_transactions (transaction_id, merchant_category, transaction_status, transaction_date)
VALUES
    (1, 'Retail', 'SUCCESS', '2024-01-05'),
    (2, 'Retail', 'FAILED', '2024-01-10'),
    (3, 'Retail', 'SUCCESS', '2024-01-20'),
    (4, 'Dining', 'SUCCESS', '2024-01-15'),
    (5, 'Dining', 'FAILED', '2024-01-18'),
    (6, 'Entertainment', 'SUCCESS', '2024-01-25'),
    (7, 'Entertainment', 'SUCCESS', '2024-01-30'),
    (8, 'Groceries', 'FAILED', '2024-01-12'),
    (9, 'Groceries', 'FAILED', '2024-01-27'),
    (10, 'Utilities', 'SUCCESS', '2024-01-09'),
    (11, 'Retail', 'SUCCESS', '2024-02-03'),
    (12, 'Dining', 'SUCCESS', '2024-02-08'),
    (13, 'Entertainment', 'FAILED', '2024-02-14'),
    (14, 'Groceries', 'SUCCESS', '2024-02-10'),
    (15, 'Utilities', 'FAILED', '2024-02-05'),
    (16, 'Retail', 'FAILED', '2024-03-12'),
    (17, 'Dining', 'FAILED', '2024-03-20'),
    (18, 'Entertainment', 'SUCCESS', '2024-03-03'),
    (19, 'Groceries', 'SUCCESS', '2024-03-29'),
    (20, 'Utilities', 'SUCCESS', '2024-03-15'),
    (21, 'Travel', 'SUCCESS', '2024-01-11'),
    (22, 'Travel', 'SUCCESS', '2024-01-19'),
    (23, 'Travel', 'SUCCESS', '2024-02-04'),
    (24, 'Travel', 'SUCCESS', '2024-02-11'),
    (25, 'Travel', 'SUCCESS', '2024-03-22');

SELECT * FROM fct_transactions;
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_transaction = pd.read_csv('Data/016/fct_transactions.csv')

df_transaction.head()

,transaction_id,merchant_category,transaction_status,transaction_date
0,1,Retail,SUCCESS,2024-01-05
1,2,Retail,FAILED,2024-01-10
2,3,Retail,SUCCESS,2024-01-20
3,4,Dining,SUCCESS,2024-01-15
4,5,Dining,FAILED,2024-01-18


# Pregunta 1

### Para enero de 2024, ¿cuáles son los recuentos totales de transacciones exitosas y fallidas en cada categoría de comercio? Este análisis ayudará al equipo de seguridad de Google Pay a identificar posibles puntos de fricción en el procesamiento de pagos.

In [4]:
df_enero = df_transaction[df_transaction['transaction_date'].between('2024-01-01','2024-01-31')]

respuesta1 = df_enero.groupby(['merchant_category','transaction_status'])['transaction_id'].count().reset_index(name='total_transaction')

respuesta1

,merchant_category,transaction_status,total_transaction
0,Dining,FAILED,1
1,Dining,SUCCESS,1
2,Entertainment,SUCCESS,2
3,Groceries,FAILED,2
4,Retail,FAILED,1
5,Retail,SUCCESS,2
6,Travel,SUCCESS,2
7,Utilities,SUCCESS,1


```SQL
SELECT
    merchant_category,
    transaction_status,
    COUNT(*) AS total_transactions
FROM fct_transactions
WHERE transaction_date BETWEEN  '2024-01-01' AND '2024-01-31'
GROUP BY merchant_category, transaction_status;
```

# Pregunta 2

### Para el primer trimestre de 2024, ¿qué categorías de comercios registraron una tasa de éxito de transacciones inferior al 90%? Esta información guiará nuestra priorización de mejoras de seguridad para aumentar la confiabilidad de los pagos.

In [5]:
df_q1 = df_transaction[df_transaction['transaction_date'].between('2024-01-01','2024-03-31')]

reporte = df_q1.groupby('merchant_category')['transaction_status'].agg(
    success_rate = lambda x: (x == 'SUCCESS').mean() * 100
).reset_index()

resultado_final = reporte[reporte['success_rate'] < 90]

resultado_final

,merchant_category,success_rate
0,Dining,50.000000
1,Entertainment,75.000000
2,Groceries,50.000000
3,Retail,60.000000
5,Utilities,66.666667


```SQL
SELECT
    merchant_category,
    COUNT(CASE WHEN transaction_status = 'SUCCESS' THEN 1 END) * 100.0 / COUNT(*) AS success_rate
FROM fct_transactions
WHERE transaction_date BETWEEN '2024-01-01' AND '2024-03-31'
GROUP BY merchant_category
HAVING (COUNT(CASE WHEN transaction_status = 'SUCCESS' THEN 1 END) * 100.0 / COUNT(*)) < 90;
```

# Pregunta 3

### Del 1 de enero al 31 de marzo de 2024, ¿podrías generar una lista de categorías de comercios con sus conteos concatenados de transacciones exitosas y fallidas? Luego, clasifica las categorías por volumen total de transacciones. Este ranking apoyará nuestra evaluación de las áreas donde los resultados mixtos de transacciones pueden afectar la experiencia del usuario.

In [7]:
# 1. Filtrar por rango de fecha
df_q1 = df_transaction[df_transaction['transaction_date'].between('2024-01-01', '2024-03-31')]

# 2. Agregación condicional
resumen = df_q1.groupby('merchant_category').agg(
    success_count=('transaction_status', lambda x: (x == 'SUCCESS').sum()),
    failed_count=('transaction_status', lambda x: (x == 'FAILED').sum()),
    total_volume=('transaction_id', 'count')
)

# 3. Concatenar en un solo string
resumen['results_summary'] = (
    resumen['success_count'].astype(str) + " Success / " + 
    resumen['failed_count'].astype(str) + " Failed"
)

# 4. Ordenar por volumen total (Ranking)
resultado_final = resumen[['results_summary', 'total_volume']].sort_values(by='total_volume', ascending=False)

resultado_final

,results_summary,total_volume
merchant_category,,
Travel,5 Success / 0 Failed,5
Retail,3 Success / 2 Failed,5
Entertainment,3 Success / 1 Failed,4
Dining,2 Success / 2 Failed,4
Groceries,2 Success / 2 Failed,4
Utilities,2 Success / 1 Failed,3


```SQL
SELECT 
    merchant_category,
    CONCAT(
        COUNT(CASE WHEN transaction_status = 'SUCCESS' THEN 1 END), 
        ' Success / ', 
        COUNT(CASE WHEN transaction_status = 'FAILED' THEN 1 END), 
        ' Failed'
    ) AS results_summary,
    COUNT(*) AS total_volume
FROM fct_transactions
WHERE transaction_date BETWEEN '2024-01-01' AND '2024-03-31'
GROUP BY merchant_category
ORDER BY total_volume DESC;
```